In [17]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
import statsmodels.api as sm
from statsmodels.graphics.gofplots import qqplot


from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet

import optuna
import xgboost as xgb

import warnings
pd.set_option('display.max_columns', None)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter('ignore', pd.errors.SettingWithCopyWarning)
warnings.simplefilter(action="ignore", category=pd.errors.PerformanceWarning)


In [18]:
train_wOrdinal = pd.read_csv('~/Documents/AmesHousingML/data_cleaning/data_ordinal.csv') 
train_originalCleaned = pd.read_csv('~/Documents/AmesHousingML/data_cleaning/data_original.csv') 
train_wOnlyDummies = pd.read_csv('~/Documents/AmesHousingML/data_cleaning/data_dummies.csv') 
train_wOnlyDummiesNoDrop = pd.read_csv('~/Documents/AmesHousingML/data_cleaning/data_dummies_noDrop.csv') 

In [19]:
train_wOnlyDummiesNoDrop

,GrLivArea,SalePrice,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,MoSold,YrSold,MSZoning_A (agr),MSZoning_C (all),MSZoning_FV,MSZoning_I (all),MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_NoAlley,Alley_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_AllPub,Utilities_NoSewr,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_Greens,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Artery,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_PreCast,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsbShng,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Plywood,Exterior2nd_PreCast,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Ex,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Ex,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtQual_Ex,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_NoBsmt,BsmtQual_Po,BsmtQual_TA,BsmtCond_Ex,BsmtCond_Fa,BsmtCond_Gd,BsmtCond_NoBsmt,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Av,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtExposure_NoBsmt,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_NoBsmt,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_ALQ,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_NoBsmt,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,HeatingQC_Ex,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_N,CentralAir_Y,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_SBrkr,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_Po,KitchenQual_TA,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sal,Functional_Typ,FireplaceQu_Ex,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_NoFireplace,FireplaceQu_Po,FireplaceQu_TA,GarageType_2Types,GarageType_Attchd,

In [20]:
# Create a KFold object
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [31]:
masterScores = pd.DataFrame(columns = ['Model', 'data_process', 'r2_5kf_mean', 'rmse_5kf_mean'])

In [22]:
ourFrames = [train_wOrdinal, train_wOnlyDummies, train_wOnlyDummiesNoDrop]
outlierType = ['threeXOutlier', 'onlyNormalCondition', 'allOutliers']

In [23]:
# xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

In [32]:
for dataframe in ourFrames:
    dataset = 'NaN'
    if (dataframe.equals(train_wOrdinal)):
        dataset = 'ordinal'
    elif (dataframe.equals(train_wOnlyDummies)):
        dataset = 'dummified'
    elif (dataframe.equals(train_wOnlyDummiesNoDrop)):
        dataset = 'dummifiedNoDrop'
        
    for trial in outlierType:
        #this will create the model name by which outlier type, below we will adjust the dataset
        # to match the outlier type being tested, it will then be stored by dataset_outlierType
        model_name = 'xgboost'
        data_process = f"{dataset}_{trial}"
        
        #below is filtering out the 3*IQR range if the trial is for that purpose
        if (trial == 'threeXOutlier'):
            frame = dataframe.copy()
            outlier_threshold = 3
            Q1 = frame['SalePrice'].quantile(.25)
            Q3 = frame['SalePrice'].quantile(.75)
            IQR = Q3 - Q1
            new_bounds = Q3 + outlier_threshold * IQR
            frame.drop(frame[frame['SalePrice'] > new_bounds].index, axis=0, inplace = True)
            
        #below is filtering out the houses that sold in normal condition
        elif (trial == 'onlyNormalCondition'):
            frame = dataframe.copy()
            if (frame.equals(train_wOrdinal)):
                frame = frame[frame['SaleCondition'] == 4]
            elif (frame.equals(train_wOnlyDummies)):
                frame = frame[frame['SaleCondition_Normal'] == 1.0]
            elif (frame.equals(train_wOnlyDummiesNoDrop)):
                frame = frame[frame['SaleCondition_Normal'] == 1.0]
                
        elif (trial == 'allOutliers'):
            frame = dataframe.copy()
        
    
        #the features will be X (independent variables)
        X = frame.drop('SalePrice', axis=1)
        X_array = X.values
        #the target (dependent variable) will be y
        y = frame['SalePrice']
        y_array = y.values

        #Split your training and testing sets of data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)


        # Train the model on the training set
        xgb_model.fit(X_train, y_train)

        # Make predictions on the test set
        xgb_model_y_pred = xgb_model.predict(X_test)

       
        # Display feature importances
        feature_importances = pd.DataFrame({'Feature': X.columns, 'Importance': xgb_model.feature_importances_})
        feature_importances = feature_importances.sort_values(by='Importance', ascending=False)
        print('\nFeature Importances:')
        print(feature_importances)
        
        # Perform cross-validation
        cv_scores = cross_val_score(xgb_model, X, y, cv=kf, scoring='r2')
        
        def rmse_scorer(y_true, y_pred):
            mse = mean_squared_error(y_true, y_pred)
            rmse = np.sqrt(mse)
            return rmse
    
        # Define the scoring function using neg_mean_squared_error
        scorer = make_scorer(rmse_scorer)
        
        # Use cross_val_score with the defined scorer
        rmse_scores = cross_val_score(xgb_model, X, y, cv=kf, scoring=scorer)
        
        # Display the cross-validation scores
        print("Cross-Validation Scores:", cv_scores, '\n')
        print(f"Mean R^2: {cv_scores.mean()}", '\n') 
        
      
        record = {'Model': model_name, 'data_process': data_process, 'r2_5kf_mean': cv_scores.mean(), 'rmse_5kf_mean': rmse_scores.mean()}
        masterScores = masterScores.append(record, ignore_index=True)

        print('\n')
        print('\n')
        print('\n')



Feature Importances:
          Feature  Importance
17    OverallQual    0.244044
75   TotalHouseSF    0.204901
80  HighQualFinSF    0.190905
52    KitchenQual    0.063921
55     Fireplaces    0.032315
..            ...         ...
39        Heating    0.000032
22       RoofMatl    0.000000
14     Condition2    0.000000
9       Utilities    0.000000
5          Street    0.000000

[84 rows x 2 columns]
Cross-Validation Scores: [0.90716289 0.90308633 0.91244088 0.91718626 0.87065985] 

Mean R^2: 0.9021072451249177 








Feature Importances:
          Feature  Importance
80  HighQualFinSF    0.293481
75   TotalHouseSF    0.235833
17    OverallQual    0.178690
41     CentralAir    0.032869
51   KitchenAbvGr    0.026730
..            ...         ...
62     GarageQual    0.000012
14     Condition2    0.000002
45   LowQualFinSF    0.000000
74  SaleCondition    0.000000
9       Utilities    0.000000

[84 rows x 2 columns]
Cross-Validation Scores: [0.92861163 0.92901212 0.89299391 0.91639739

In [25]:
train_wOrdinal.head()

,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,Fence,MoSold,YrSold,SaleType,SaleCondition,TotalHouseSF,TotalBathroomCount,QualityOutdoorSF,YearAndRemodAvg,NonHouseSF,HighQualFinSF,HouseLotRatio,FrontageLotRatio,QualityOutdoorLotRatio
0,856,126000,30,5,60.0,7890,1,1,3,3,0,0,0,19,2,2,0,2,6,6,1939,1950,1,0,13,14,2,0.0,3,3,1,4,4,2,3,238.0,1,0.0,618.0,856.0,1,3,1,3,856,0,0,1.0,0.0,1,0,2,1,3,4,6,1,5,5,1939.0,1,2.0,399.0,4,4,2,0,0,0,0,166,1,3,2010,9,4,1712.0,2.0,166,1944.5,6178.0,1712.0,21.698352,0.760456,2.103929
1,1049,139500,120,5,42.0,4235,1,1,3,3,0,4,0,7,2,2,4,2,5,5,1984,1984,1,0,6,6,1,149.0,4,3,1,5,4,3,3,552.0,2,393.0,104.0,1049.0,1,3,1,3,1049,0,0,1.0,0.0,2,0,2,1,4,5,6,0,1,1,1984.0,3,1.0,266.0,4,4,2,0,105,0,0,0,1,2,2009,9,4,2098.0,3.0,105,1984.0,2137.0,2098.0,49.539551,0.991736,2.479339
2,1001,124900,30,1,60.0,6060,1,1,3,3,0,4,0,10,2,2,0,2,5,9,1930,2007,3,0,8,8,2,0.0,4,3,0,4,4,2,2,737.0,1,0.0,100.0,837.0,1,5,1,3,1001,0,0,0.0,0.0,1,0,2,1,4,5,6,0,1,5,1930.0,1,1.0,216.0,4,2,0,154,0,42,86,0,1,11,2007,9,4,1838.0,1.0,282,1968.5,4222.0,1838.0,30.330033,0.990099,4.653465
3,1039,114000,70,5,80.0,8146,1,1,3,3,0,0,0,18,2,2,0,5,4,8,1900,2003,1,0,8,8,2,0.0,4,4,0,3,4,2,1,0.0,1,0.0,405.0,405.0,1,4,1,3,717,322,0,0.0,0.0,1,0,2,1,3,6,6,0,1,5,1940.0,1,1.0,281.0,4,4,0,0,0,168,0,111,1,5,2009,9,4,1444.0,1.0,279,1951.5,6702.0,1444.0,17.726492,0.982077,3.424994
4,1665,227000,60,5,70.0,8400,1,1,3,3,0,4,0,15,2,2,0,5,8,6,2001,2001,1,0,12,13,2,0.0,4,3,2,5,4,2,3,643.0,1,0.0,167.0,810.0,1,5,1,3,810,855,0,1.0,0.0,2,1,3,1,4,6,6,0,1,1,2001.0,3,2.0,528.0,4,4,2,0,45,0,0,0,1,11,2009,9,4,2475.0,3.5,45,2001.0,5925.0,2475.0,29.464286,0.833333,0.535714


In [33]:
for dataframe in ourFrames:
    dataset = 'NaN'
    if (dataframe.equals(train_wOrdinal)):
        dataset = 'ordinal'
    elif (dataframe.equals(train_wOnlyDummies)):
        dataset = 'dummified'
    elif (dataframe.equals(train_wOnlyDummiesNoDrop)):
        dataset = 'dummifiedNoDrop'
        
    for trial in outlierType:
        #this will create the model name by which outlier type, below we will adjust the dataset
        # to match the outlier type being tested, it will then be stored by dataset_outlierType
        model_name = 'xgboost_tuned'
        data_process = f"{dataset}_{trial}"
        
        #below is filtering out the 3*IQR range if the trial is for that purpose
        if (trial == 'threeXOutlier'):
            frame = dataframe.copy()
            outlier_threshold = 3
            Q1 = frame['SalePrice'].quantile(.25)
            Q3 = frame['SalePrice'].quantile(.75)
            IQR = Q3 - Q1
            new_bounds = Q3 + outlier_threshold * IQR
            frame.drop(frame[frame['SalePrice'] > new_bounds].index, axis=0, inplace = True)
            
        #below is filtering out the houses that sold in normal condition
        elif (trial == 'onlyNormalCondition'):
            frame = dataframe.copy()
            if (frame.equals(train_wOrdinal)):
                frame = frame[frame['SaleCondition'] == 4]
            elif (frame.equals(train_wOnlyDummies)):
                frame = frame[frame['SaleCondition_Normal'] == 1.0]
            elif (frame.equals(train_wOnlyDummiesNoDrop)):
                frame = frame[frame['SaleCondition_Normal'] == 1.0]
                
        elif (trial == 'allOutliers'):
            frame = dataframe.copy()
        
    
        #the features will be X (independent variables)
        X = frame.drop('SalePrice', axis=1)
        X_array = X.values
        #the target (dependent variable) will be y
        y = frame['SalePrice']
        y_array = y.values

        #Split your training and testing sets of data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        
        params =  {'max_depth': 3,
                         'learning_rate': 0.0836719064919561,
                         'n_estimators': 629,
                         'min_child_weight': 2,
                         'colsample_bytree': 0.2015968025244805,
                         'subsample': 0.7345072578143232,
                         'reg_alpha': 0.9256222240627585,
                         'reg_lambda': 0.16575971012532154}
        
        
        
        # params =  {'objective': 'reg:squarederror',
        #            'booster': 'gbtree',
        #              'max_depth': 3,
        #              'learning_rate': 0.056842782261922145,
        #              'n_estimators': 900,
        #               'random_state': 42}
        
        xgb_model = xgb.XGBRegressor(**params)


        # Train the model on the training set
        xgb_model.fit(X_train, y_train)

        # Make predictions on the test set
        xgb_model_y_pred = xgb_model.predict(X_test)

       
        # Display feature importances
        feature_importances = pd.DataFrame({'Feature': X.columns, 'Importance': xgb_model.feature_importances_})
        feature_importances = feature_importances.sort_values(by='Importance', ascending=False)
        print('\nFeature Importances:')
        print(feature_importances)
        
        # Perform cross-validation
        cv_scores = cross_val_score(xgb_model, X, y, cv=kf, scoring='r2')
        
        def rmse_scorer(y_true, y_pred):
            mse = mean_squared_error(y_true, y_pred)
            rmse = np.sqrt(mse)
            return rmse
    
        # Define the scoring function using neg_mean_squared_error
        scorer = make_scorer(rmse_scorer)
        
        # Use cross_val_score with the defined scorer
        rmse_scores = cross_val_score(xgb_model, X, y, cv=kf, scoring=scorer)
        
        # Display the cross-validation scores
        print("Cross-Validation Scores:", cv_scores, '\n')
        print(f"Mean R^2: {cv_scores.mean()}", '\n') 
        
      
        record = {'Model': model_name, 'data_process': data_process, 'r2_5kf_mean': cv_scores.mean(), 'rmse_5kf_mean': rmse_scores.mean()}
        masterScores = masterScores.append(record, ignore_index=True)

        print('\n')
        print('\n')
        print('\n')



Feature Importances:
               Feature  Importance
17         OverallQual    0.185389
75        TotalHouseSF    0.078123
80       HighQualFinSF    0.070498
76  TotalBathroomCount    0.055566
56         FireplaceQu    0.050462
..                 ...         ...
10           LotConfig    0.000872
60          GarageCars    0.000824
39             Heating    0.000524
47        BsmtHalfBath    0.000340
9            Utilities    0.000000

[84 rows x 2 columns]
Cross-Validation Scores: [0.93970221 0.92135197 0.93808568 0.9315053  0.87389578] 

Mean R^2: 0.9209081890389313 








Feature Importances:
               Feature  Importance
17         OverallQual    0.162393
80       HighQualFinSF    0.094320
75        TotalHouseSF    0.078091
76  TotalBathroomCount    0.077375
52         KitchenQual    0.049543
..                 ...         ...
45        LowQualFinSF    0.000487
42          Electrical    0.000482
5               Street    0.000313
74       SaleCondition    0.000000
9      

In [27]:
# # {'booster': 'gbtree',
#  'max_depth': 3,
#  'learning_rate': 0.056842782261922145,
#  'n_estimators': 900}

In [34]:
masterScores #want to use the sci kit and catboost

,Model,data_process,r2_5kf_mean,rmse_5kf_mean
0,xgboost,ordinal_threeXOutlier,0.902107,20866.299829
1,xgboost,ordinal_onlyNormalCondition,0.919789,19881.826863
2,xgboost,ordinal_allOutliers,0.904445,23142.592485
3,xgboost,dummified_threeXOutlier,0.902148,20859.072894
4,xgboost,dummified_onlyNormalCondition,0.918049,20111.383128
5,xgboost,dummified_allOutliers,0.896948,24052.156077
6,xgboost,dummifiedNoDrop_threeXOutlier,0.900806,20997.812741
7,xgboost,dummifiedNoDrop_onlyNormalCondition,0.917838,20159.872704
8,xgboost,dummifiedNoDrop_allOutliers,0.904017,23207.145705
9,xgboost_tuned,ordinal_threeXOutlier,0.920908,18584.941593


In [35]:
output_file = 'masterScores_xgboost.csv'

masterScores.to_csv(output_file, index=False, mode='w')

print(f"DataFrame exported to '{output_file}' in the same working directory.")

DataFrame exported to 'masterScores_xgboost.csv' in the same working directory.


In [30]:
c + a 

NameError: name 'c' is not defined

In [ ]:
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    max_depth = trial.suggest_int('max_depth', 10, 50)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 32)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 32)
    
    model = RandomForestRegressor(n_estimators = n_estimators, 
                                  max_depth = max_depth, 
                                  min_samples_split = min_samples_split, 
                                  min_samples_leaf = min_samples_leaf)
    
    score = cross_val_score(model, X_train, y_train, cv = 5, scoring = 'neg_mean_squared_error', n_jobs = -1).mean()
    
    return score

In [ ]:
# study = optuna.create_study(direction = 'maximize', sampler = optuna.samplers.RandomSampler(seed=42))

In [ ]:
# study.optimize(objective, n_trials = 100)

In [ ]:
# study.best_params

In [ ]:
# best_params = study.best_params

In [ ]:
# optuna.visualization.plot_optimization_history(study)

In [ ]:
# optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
# optuna.visualization.plot_slice(study,params=['n_estimators', 'max_depth', 'min_samples_split', 'min_samples_leaf'])

In [ ]:
# optuna.visualization.plot_param_importances(study)

In [ ]:
# best_n_estimators = best_params['n_estimators']
# best_max_depth = best_params['max_depth']
# best_min_samples_split = best_params['min_samples_split']
# best_min_samples_leaf = best_params['min_samples_leaf']



In [ ]:
# best_model = RandomForestRegressor(n_estimators=best_n_estimators,
#                                    max_depth=best_max_depth,
#                                    min_samples_split=best_min_samples_split,
#                                    min_samples_leaf=best_min_samples_leaf)

In [ ]:
# best_model.fit(X_train,y_train)

In [ ]:
# y_pred = best_model.predict(X_test)

In [ ]:
# mean_squared_error(y_test, y_pred, squared = False)

In [ ]:
# r2_score(y_test, y_pred)

In [ ]:

def objective(trial):
    params = {
        'objective': 'reg:squarederror',
        'booster': trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart']),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=100),
    }

    model = xgb.XGBRegressor(**params, random_state=42)
    model.fit(X_train, y_train)

    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    return mse

#study = optuna.create_study(direction='minimize')
#study.optimize(objective, n_trials=100)

best_params = study.best_params
best_model = xgb.XGBRegressor(**best_params, random_state=42)
best_model.fit(X_train, y_train)

In [ ]:
best_params

In [ ]:
best_model

In [ ]:
best_pred = best_model.predict(X_test)

In [ ]:
model_name = 'xgboost_tuned'
#Split your training and testing sets of data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Evaluate the Random Forest model
best_r2 = r2_score(y_test, best_pred)
print(f'R-squared on the test set: {best_r2}')

best_model_mse = mean_squared_error(y_test, best_pred)
print(f'Mean Squared Error on the test set: {best_model_mse}')




# Create a KFold object
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation
cv_scores = cross_val_score(best_model, X, y, cv=kf, scoring='r2')
# You can replace 'r2' with other scoring metrics like 'neg_mean_squared_error', etc.

    

# Use cross_val_score with the defined scorer
rmse_scores = cross_val_score(best_model, X, y, cv=kf, scoring=scorer)



# Display the cross-validation scores
print("Cross-Validation Scores Rsquared:", cv_scores, '\n')


# Print the mean and standard deviation of the scores
print(f"Mean R^2: {cv_scores.mean()}", '\n')
print(f"Standard Deviation R^2: {cv_scores.std()}", '\n')

record = {'Model': model_name, 'r2_5kf_mean': cv_scores.mean(), 'rmse_5kf_mean': rmse_scores.mean()}
masterScores = masterScores.append(record, ignore_index=True)


In [ ]:
import shap

# Use the SHAP explainer
explainer = shap.Explainer(best_model)

# Calculate SHAP values for a subset of the data (e.g., test set)
shap_values = explainer.shap_values(X_test)

# Plot summary plot
shap.summary_plot(shap_values, X_test)

In [ ]:
masterScores

In [ ]:
output_file = 'masterScores_xgboost.csv'

masterScores.to_csv(output_file, index=False, mode='w')

print(f"DataFrame exported to '{output_file}' in the same working directory.")